In [52]:
import pandas as pd
import numpy as np

In [21]:
pe = pd.read_csv("historical_pe_ratio.csv")
pe.set_index("Unnamed: 0", inplace=True)
price = pd.read_csv("data/sp500_close_data.csv")
price.set_index("Date", inplace=True)

In [47]:
data = []
dates = pe.index
price.index = pd.to_datetime(price.index)

for j in dates:
    try:
        row = price.loc[j]
    except KeyError:
        closest_date = price.index[min(range(len(price.index)), key=lambda i: abs(price.index[i] - pd.to_datetime(j)))]
        row = price.loc[closest_date]

    data.append(row)

In [63]:
df = pd.DataFrame(np.array(data), columns=pe.columns)
df.index = (pd.date_range(start='2014-07-01', end='2024-04-01', freq='Q').strftime('%Y-%m-%d').tolist())[::-1]


In [84]:
df1_numeric = pd.to_numeric(df.stack(), errors='coerce').unstack()
df2_numeric = pd.to_numeric(pe.stack(), errors='coerce').unstack()

result = df1_numeric.div(df2_numeric)

In [85]:
for col in result.columns:
    if col != 'Year':
        result[col] = result[col].fillna(result[col].shift(1).rolling(window=3, min_periods=1).mean())

In [87]:
result.to_csv("historical_eps.csv")